# Final Project: Plasma Sheath Formation

## Introduction

Plasma is the fourth state of matter; it forms when a gas is heated to such a degree that electrons become freed from their atoms.  It makes up over 99% of the observable universe and is relevant to a range of scientific and consumer applications. A few examples of plasma and its applications are[1]:
- Lightning
- Stars
- Aurora
- Fluorescent lightbulbs
- Plasma televisions
- Fusion reactors

For my project, I wrote a program that simulates an electrically neutral, collisionless plasma at thermal equilibrium in a vacuum, with the goal of demonstrating the formation of a plasma sheath. Because electrons are much lighter than ions, they have a higher initial speed within the plasma, which allows some electrons to escape, leaving behind a region with a net positive charge. This boundary region is called the sheath, where the charge imbalance results in an electric field that pushes ions out of the plasma while keeping electrons from escaping. The sheath regulates the rates at which the particles leave the plasma, with the rates eventually reaching an equilibrium[2]. Understanding the physics of plasma sheaths is important for applications such as fusion reactors and plasma propulsion engines.

In this program, I focused on showing the formation of a sheath in a visually obvious way using Matplotlib figures. When considering project ideas, I knew I wanted to do a project with a visually appealing result, and I found [this](https://www.youtube.com/watch?v=SJUyCjAwrTI) video of a sheath simulation, which I thought looked really nice while also covering a topic I was interested in, so I decided to see if I could replicate the simulation[3]. My program uses the Particle-in-Cell (PIC) method to simulate the plasma, with calculations for the electric potential and particle motion performed using the Finite Difference Method (FDM) and Verlet integration. The final result is a GIF showing the distributions of particles and the structure of the electric field in the plasma.

## Methods

The PIC method is the standard computational method used to simulate the motion of charged particles, so I immediately settled on using it to produce my simulation. My program involves a few main steps: first, it initializes the particles by calculating their initial velocities and arranging them uniformly in a disk; next, the step that the name particle-in-cell comes from, it calculates the charge density at each node of the grid that the simulation space is divided into; then, using the charge density, it calculates the electric potential at each node using Poisson's equation, which is solved using the Finite Difference Method (FDM); the electric field is then calculated from the potential and interpolated back onto the particle positions; next, the electric field is used to calculate each particle's acceleration, and the particle's new position, velocity, and acceleration are calculated using Verlet integration; finally, the Matplotlib package is used to generate a figure visualizing the plasma's behavior[4].

The first step of my program's main section is shown below. This is the first function called in the main simulation loop, because it creates the particles and generates their starting positions and velocities. The positions are generated from a random number generator and placed within a circle centered at the center of the simulation box. The directional components of velocity for particles in a plasma at thermal equilibrium follow a normal distribution, so the particle velocities can simply be generated from a normal distribution, with the standard deviation sigma calculated as shown below, where $k_B$ is the Boltzmann constant, T is the temperature of the particles in the plasma, and m is the particle mass[5][6]:
$$\sigma = \sqrt{\frac{k_B T}{m}}$$
The components for position and velocity are placed into an n $\times$ 2 numpy array, where n is the number of particles being simulated. When calculating the initial distance of each particle from the center, the square root of the random number is taken to avoid clumping the particles toward the center of the plasma disk[7].

In [ ]:
# Initializing particles
def initialize_particles():
    '''Generates initial particle positions and velocities and distributes them uniformly in a circle of radius R_plasma'''
    # Calculating standard deviation for velocity initialization
    sig_e = np.sqrt(kB * T_plasma / m_e_real)
    sig_p = np.sqrt(kB * T_plasma / m_p_real)
    # Generating empty pos and vel arrays
    pos_e = np.zeros((N_particles, 2))
    vel_e = np.zeros((N_particles, 2))
    # Generating distance from center of plasma for each particle
    r = R_plasma * np.sqrt(np.random.rand(N_particles))
    # Generating angle at which each particle is placed relative to center of plasma
    theta = 2 * np.pi * np.random.rand(N_particles)
    # Converting the particle positions to cartesian coordinates and placing them relative to the center of the simulation box
    pos_e[:, 0] = L/2 + r * np.cos(theta)
    pos_e[:, 1] = L/2 + r * np.sin(theta)
    # Generating initial velocity components independently for each particle from Gaussian distribution with mean=0 (no bulk motion), sigma from previous calculation
    vel_e[:, 0] = np.random.normal(0, sig_e, N_particles)
    vel_e[:, 1] = np.random.normal(0, sig_e, N_particles)

    # Repeating steps from above for other particle species (protons)
    pos_p = np.zeros((N_particles, 2))
    vel_p = np.zeros((N_particles, 2))
    r = R_plasma * np.sqrt(np.random.rand(N_particles))
    theta = 2 * np.pi * np.random.rand(N_particles)
    pos_p[:, 0] = L/2 + r * np.cos(theta)
    pos_p[:, 1] = L/2 + r * np.sin(theta)
    vel_p[:, 0] = np.random.normal(0, sig_p, N_particles)
    vel_p[:, 1] = np.random.normal(0, sig_p, N_particles)

    return pos_e, vel_e, pos_p, vel_p

The next step in the PIC algorithm, done with the code shown below, is to assign the charge of the individual particles to their nearest grid point and calculate the charge density ($\rho$) at each grid point. In my program, I create an Nx $\times$ Nx array to store the charge value at each node on the grid, where Nx is the number of grid cells along each axis. I convert the position coordinates for each particle to grid coordinates using the formula $x_g = \frac{x}{dx - 0.5}$, where x is the particle coordinate on the x-axis and dx is the size of each grid cell. Taking the floor of these values gives me the index of the lower-left node of the cell containing each particle. I then calculate the weight for each node based on the particle's distance from the node and create a mask to remove any particles that have left the simulation area or are directly on the boundaries from the density calculation. Finally, I use the np.add.at function to distribute the charge from each particle onto its surrounding four grid nodes based on the weight assigned to each node. The function returns an array containing the charge density at each node.

In [ ]:
# Grid assignment and charge density
def calculate_density(pos, q_eff):
    '''Assigns the charge of each particle to the nearest node; calculates rho at each node
       Inputs:
        pos: particle positions array
        q_eff: the weighted charge of the each macroparticle
       Returns: array of charge density values for each node
    '''
    # Creating Nx x Nx array to store the charge value at each node
    density = np.zeros((Nx, Nx))
    # Converting particle positions to grid coordinates, assigns particle to the lower-left node of the cell containing the particle
    x_norm = pos[:, 0] / dx - 0.5
    y_norm = pos[:, 1] / dx - 0.5
    # Obtaining integer indices for the node to the lower-left of the particle
    i = np.floor(x_norm).astype(int)
    j = np.floor(y_norm).astype(int)
    # Creating weighting terms for each particle by calculating distance from particle to its node
    wx = 1 - (x_norm - i)
    wy = 1 - (y_norm - j)
    # Checking that the indices for each particle are within the grid, particles that are outside the simulation area or directly on the edge are excluded from the density calculation
    mask = (i >= 0) & (i < Nx-1) & (j >= 0) & (j < Nx-1)
    i = i[mask]; j = j[mask]; wx = wx[mask]; wy = wy[mask]
    # Distributing each particle's charge to the four surrounding nodes
    np.add.at(density, (i, j), q_eff * wx * wy)
    np.add.at(density, (i+1, j), q_eff * (1-wx) * wy)
    np.add.at(density, (i, j+1), q_eff * wx * (1-wy))
    np.add.at(density, (i+1, j+1), q_eff * (1-wx) * (1 - wy))

    return density / (dx**2)

The next step, solving the Poisson equation using the Finite Difference Method for the electric potential at each point on the grid, involves two functions: build_matrix and solve_poisson, both shown below. The Poisson equation for the electric potential is
$$\nabla^2\phi = -\frac{\rho}{\epsilon_0},$$
which can be discretized in 2D, with the potential at each grid point (i, j) denoted $\phi_{i, j}$ , as 
$$\frac{\phi_{i+1,j}-2\phi_{i,j}+\phi_{i-1,j}}{(\Delta x)^2} + \frac{\phi_{i, j+1}-2\phi_{i,j}+\phi_{i, j-1}}{(\Delta y)^2} = -\frac{\rho_{i,j}}{\epsilon_0}$$
Because my simulation grid is uniform, with $\Delta x = \Delta y = dx$, the equation simplifies to
$$\phi_{i-1,j}+\phi_{i+1,j}+\phi_{i,j-1}+\phi_{i,j+1}-4\phi_{i,j} = -\frac{\rho_{i,j}}{\epsilon_0}dx^2.$$
I then use the scipy.sparse package to convert problem into a matrix equation, $A\vec{\phi} = \vec{b}$, where $A$ is the coefficient matrix representing the Poisson equation for each grid point, $\vec{\phi}$ is the vector of unknowns (the electric potential at each node), and $\vec{b}$ is the vector containing the known charge densities at each node. The solve_poisson equation then builds the $\vec{b}$ vector and uses the spsolve function from scipy.sparse to solve the matrix equation $A\vec{\phi} = \vec{b}$ for the electric potential at each node, returning the data in an Nx $\times$ Nx array.

In [ ]:
# Building matrix for Poisson solver and enforcing phi=0 at the edges of the simulation box
def build_matrix(nx):
    '''
    Builds the matrix A, which represents the Laplacian operator in the Poisson equation
       Inputs:
        nx: the number of grid points per axis
       Returns:
        A: the matrix representing the Laplacian operator
    '''
    # Length of diag arrays (number of unknowns in system of equations)
    N = nx * nx
    # Forming diagonals based on coefficients from discretized Poisson equation (shown in notebook)
    main_diag = -4 * np.ones(N)
    off_diag_x = np.ones(N - 1)
    off_diag_y = np.ones(N - nx)
    # Preventing flattened array from wrapping matrix rows into each other.
    for i in range(1, N):
        if i % nx == 0:
            off_diag_x[i-1] = 0
    # Building the sparse matrix
    diagonals = [main_diag, off_diag_x, off_diag_x, off_diag_y, off_diag_y]
    offsets = [0, -1, 1, -nx, nx]
    A = diags(diagonals, offsets, shape=(N, N), format='csc')
    return A

matrix = build_matrix(Nx)

# Poisson solver
def solve_poisson(rho):
    '''
    Solves Poisson equation with phi = 0 at simulation walls in the form of a system of linear equations A*phi = b
       Input:
        rho: Nx x Nx array containing the charge density at each point on the grid
       Returns:
        phi: Nx x Nx array containing the electric potential at each point on the grid
    '''
    # Constructing b vector from right hand side of discretized Poisson equation
    b = -rho.flatten() / eps0 * dx**2
    # Solving the system for the potential and reshaping the flattened (1D) vector into an Nx x Nx array
    phi_flat = spsolve(matrix, b)
    phi = phi_flat.reshape((Nx, Nx))
    return phi

The next step is to calculate the electric field components from the potential, which is handled by the calculate_electric_field function shown below. This calculation is much simpler than the last, as the electric field is just the negative gradient of potential:
$$E_x = -\frac{\partial \phi}{\partial x} \text{\hspace{12pt} and \hspace{12pt}} E_y = -\frac{\partial \phi}{\partial y}.$$
These are discretized as well. For example, the equation for the x-component of the field is 
$$E_{x,i,j} = -\frac{\phi_{i+1,j}-\phi_{i-1,j}}{2dx}.$$
The function returns two arrays, one for each of the directional components of the electric field.

In [ ]:
def calculate_electric_field(phi):
    # Building arrays of same shape as phi to store E field values
    Ex = np.zeros_like(phi)
    Ey = np.zeros_like(phi)
    # Calculating E field for interior points - the slices exclude the points around the edge of the simulation to enforce the boundary condition of phi=0
    Ex[1:-1, :] = -(phi[2:, :] - phi[:-2, :]) / (2*dx)
    Ey[:, 1:-1] = -(phi[:, 2:] - phi[:, :-2]) / (2*dx)
    return Ex, Ey

The next function, interpolate_field, performs bilinear interpolation to distribute the electric field values from the grid points back to each particle[9]. It makes two arrays, Ex_p and Ey_p, containing the interpolated values for each particle, and stacks them along the y-axis, returning a single array with the electric field components for each particle.

In [ ]:
def interpolate_field(pos, Ex, Ey):
    '''Using bilinear interpolation to distribute E field values from the grid nodes back to the particles'''
    # Calculating index of grid node to the lower-left of the particle, as before
    i = np.floor(pos[:, 0] / dx - 0.5).astype(int)
    j = np.floor(pos[:, 1] / dx - 0.5).astype(int)
    # Weighting using particle distance from node
    dx_local = (pos[:, 0] / dx - 0.5) - i
    dy_local = (pos[:, 1] / dx - 0.5) - j
    # Making sure calculated indices stay within the boundaries
    i = np.clip(i, 0, Nx-2)
    j = np.clip(j, 0, Nx-2)
    # Performing bilinear interpolation: each particle's E field value is a weighted average of the values at the four surrounding grid nodes
    Ex_p = (Ex[i, j] * (1-dx_local) * (1-dy_local) + 
            Ex[i+1, j] * dx_local * (1-dy_local) + 
            Ex[i, j+1] * (1-dx_local) * dy_local +
            Ex[i+1, j+1] * dx_local * dy_local)
    
    Ey_p = (Ey[i, j] * (1-dx_local) * (1-dy_local) +
            Ey[i+1, j] * dx_local * (1-dy_local) + 
            Ey[i, j+1] * (1-dx_local) * dy_local + 
            Ey[i+1, j+1] * dx_local * dy_local)
    
    return np.stack((Ex_p, Ey_p), axis = 1)

The next, and final, step of the main part of the PIC process is to update the positions and velocities of the particles, which is done using Verlet integration. I use one function, verlet_step, which takes all the necessary data about the particles and the electric field from the previous steps and the initial conditions of the simulation, and calculates the new positions, velocities, and accelerations after one timestep, $dt$, using the Velocity Verlet algorithm[10]. It first calculates the velocity after half of a timestep:
$$v_{t+0.5dt} = v_t + \frac{a_t}{2}dt$$
Then the position $r_{t+dt}$ after a full timestep:
$$r_{t+dt} = r_{t} + v_{t+0.5dt}dt$$ 
Next, it calculates the electric field acting on the particle at its new position using the interpolate_field function, and uses that information to calculate the particle's new acceleration at time $t + dt$:
$$a_{t+dt} = \frac{q_{eff}}{m_{eff}}E_{r_{t+dt}}$$
Finally, it calculates the particle's new velocity after the full timestep using the newly calculated acceleration:
$$v_{t+dt} = v_{t+0.5dt} + \frac{a_{t+dt}}{2}dt$$
The function returns new arrays with the updated position, velocity, and acceleration values for the particles.

In [ ]:
def verlet_step(pos, vel, acc, q_eff, m_eff, Ex, Ey):
    '''Calculates the positions and velocities of the particles at each timestep using the calculated E field'''
    # Calculating the velocity at time t + 0.5dt
    vel_half = vel + 0.5 * acc * dt
    # Calculating position at time t + dt
    pos_new = pos + vel_half * dt
    # Calculating the electric field acting on the particle at its new position
    E_at_pos = interpolate_field(pos_new, Ex, Ey)
    # Calculating the new acceleration at time t + dt; uses effective (scaled) charge and mass of each particle
    acc_new = (q_eff / m_eff) * E_at_pos
    # Calculates the new velocity after the second half of the timestep (t + dt)
    vel_new = vel_half + 0.5 * acc_new * dt
    return pos_new, vel_new, acc_new

The last function in the main section of the code, remove_particles, just handles the particles that leave the simulation box by removing them. It checks the particle's x and y coordinates to make sure that they are both between $0$ and $L$ (the length of the simulation box) with a mask. The mask is applied to the position, velocity, and acceleration arrays, and any particles that do not satisfy the conditions are disabled.

In [ ]:
def remove_particles(pos, vel, acc):
    # Mask to check that particles are within simulation boundaries: must have 0 < x < L and 0 < y < L.
    mask = (pos[:, 0] > 0) & (pos[:, 0] < L) & (pos[:, 1] > 0) & (pos[:, 1] < L)
    return pos[mask], vel[mask], acc[mask]

The simulation and physical parameters are specified at the beginning of the program as shown below.

In [ ]:
# Simulation Parameters
L = 0.05                # Simulation box size (m)
Nx = 64                 # Number of grid cells per axis
dx = L / Nx             # Grid spacing
dt = 2e-11              # Time step (s)
T_plasma = 1000         # Plasma temperature (K)
R_plasma = 0.0125        # Initial radius of plasma (m)
N_particles = int(1e5)       # Number of each particle type simulated

# Physical Parameters
q_e_real = -1.602e-19   # Electron charge (C)
q_p_real = 1.602e-19    # Proton charge (C)
eps0 = 8.854e-12        # Permittivity of free space (F/m)
kB = 1.380649e-23       # Boltzmann constant (J/K)
m_e_real = 9.109e-31    # Electron mass (kg)
m_p_real = 1.672e-27    # Proton mass (kg)

The simulation loop section of the program brings all of the previously defined functions and parameters together to produce the actual simulation and visualize the results. It starts by initializing the particles and the figure the results will eventually be displayed on, and then calculating the initial state of the plasma using the initialize_particles, calculate_density, solve_poisson, calculate_electric_field, and interpolate_field functions. It also calculates the values necessary to display the plots at $t=0$. Then, the update function goes through all of the previously discussed PIC steps, updating the particle data at each timestep, until a previously defined number of timesteps (total_frames) have been calculated. The update function also updates the plots for each timestep. I also have the function print the number of frames calculated, the real time required to calculate each timestep (using Python's time package), and a log of the number of electrons and protons remaining inside the simulation boundaries. Once the loop has been completed, the animation GIF is saved, with its name determined by the current system time of the device that ran the simulation (obtained using Python's datetime package).

In [ ]:
# Simulation Loop
if __name__ == '__main__':
    # Initializing particles
    pos_e, vel_e, pos_p, vel_p = initialize_particles()
    # Initializing figure
    fig, (ax1, ax2, ax3) = pl.subplots(1, 3, figsize=(18, 5))

    # Calculating initial state
    rho_e = calculate_density(pos_e, q_e)
    rho_p = calculate_density(pos_p, q_p)

    # Calculating particle densities (m^-2)
    n_e = rho_e / q_e_real
    n_p = rho_p / q_p_real

    # Calculating electric potential and field
    phi = solve_poisson(rho_e + rho_p)
    Ex, Ey = calculate_electric_field(phi)

    # Calculating electric field magnitude for figure
    E_mag = np.sqrt(Ex**2 + Ey**2)

    acc_e = (q_e / m_e) * interpolate_field(pos_e, Ex, Ey)
    acc_p = (q_p / m_p) * interpolate_field(pos_p, Ex, Ey)

    # Making plots for figure
    # Electron density
    pl1 = ax1.imshow(n_e.T, origin='lower', extent=[0, L*100, 0, L*100], cmap='plasma', interpolation='bilinear')
    ax1.set_title('Electron Density ($m^{-2}$)')
    ax1.set_xlabel('x (cm)'); ax1.set_ylabel('y (cm)')
    pl.colorbar(pl1, ax=ax1)

    # Proton density
    pl2 = ax2.imshow(n_p.T, origin='lower', extent=[0, L*100, 0, L*100], cmap='plasma', interpolation='bilinear')
    ax2.set_title('Proton Density ($m^{-2}$)')
    ax2.set_xlabel('x (cm)'); ax2.set_ylabel('y (cm)')
    pl.colorbar(pl2, ax=ax2)

    # Electric field strength
    pl3 = ax3.imshow(E_mag.T, origin='lower', extent=[0, L*100, 0, L*100], cmap='plasma', interpolation='bilinear')
    ax3.set_title('Electric Field Strength (V/m)')
    ax3.set_xlabel('x (cm)'); ax3.set_ylabel('y (cm)')
    pl.colorbar(pl3, ax=ax3)

    # Update function to repeat steps
    def update(frame):
        t0 = time.time()
        global pos_e, vel_e, acc_e, pos_p, vel_p, acc_p

        rho_e = calculate_density(pos_e, q_e)
        rho_p = calculate_density(pos_p, q_p)
        rho_total = rho_e + rho_p

        phi = solve_poisson(rho_total)
        Ex, Ey = calculate_electric_field(phi)
        E_mag = np.sqrt(Ex**2 + Ey**2)

        pos_e, vel_e, acc_e = verlet_step(pos_e, vel_e, acc_e, q_e, m_e, Ex, Ey)
        pos_p, vel_p, acc_p = verlet_step(pos_p, vel_p, acc_p, q_p, m_p, Ex, Ey)

        pos_e, vel_e, acc_e = remove_particles(pos_e, vel_e, acc_e)
        pos_p, vel_p, acc_p = remove_particles(pos_p, vel_p, acc_p)

        # Density calculations for figure
        n_e = rho_e / q_e_real
        n_p = rho_p / q_p_real

        # Updating plots
        pl1.set_data(n_e.T)
        pl1.set_clim(vmin=0, vmax=np.max(n_e))

        pl2.set_data(n_p.T)
        pl2.set_clim(vmin=0, vmax=np.max(n_p))

        pl3.set_data(E_mag.T)
        pl3.set_clim(vmin=0, vmax=np.max(E_mag))

        dt_calc = (time.time() - t0) * 1000
        print(f'Frame {frame}/{total_frames} | Time: {dt_calc:.1f}ms | Electrons: {len(pos_e)} | Protons: {len(pos_p)}')

        return pl1, pl2, pl3
    
    ani = animation.FuncAnimation(fig, update, frames=total_frames, blit=False)

    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    ani.save(f'plasma_sim_{timestamp}.gif', writer='Pillow', fps=15)

## Results and Difficulties

![GIF of early simulation result example](simulation_20251130_130633.gif "Example of an early simulation result")

The figure shown above is one of my earlier simulation results, which shows the main issue I encountered when working on this project. From left to right, the plots show the Electron Density (electrons per $\text{m}^2$), the Proton Density ($\text{m}^{-2}$), and the strength of the electric field (V/m) at each point in the simulation space. As you can see at the beginning of the animation, instead of demonstrating the formation of a sheath, the electrons all immediately fly out into the simulation box, and the plasma immediately comes apart. Very quickly, there are no electrons left in the simulation. I tried a few different things when trying to figure out what was wrong with the simulation, mainly trying different calculation methods (Fast Fourier Transform) and boundary conditions, before I realized that what I was simulating seemed to be physically accurate, which meant there was something wrong with the conditions I was simulating. At that point, I realized that my mistake was leaving out a step in the PIC process, which I had previously thought I could avoid to make my program a bit simpler. The step I had left out was the macroparticle weighting process, where instead of simulating some number of real particles, you simulate some number of macroparticles, each of which has its charge and mass weighted so that it represents a very large number of real-world particles. This step allows you to simulate real-world conditions more accurately without slowing down your program by an extreme amount in an attempt to simulate billions or trillions of particles. The code I used to implement this change is shown below.

In [ ]:
# Macroparticle Parameters
lambda_D = 2 * dx       # Debye length - must have multiple lengths per cell to accurately portray the physics
scale_n = (eps0 * kB * T_plasma)/(lambda_D**2 * q_e_real**2) # Calculating true number of particles to simulate for accurate physics, based on calculated Debye length

plasma_area = np.pi * R_plasma**2 # Initial area of plasma (m^2)
actual_charge = scale_n * plasma_area * q_p_real # Total actual charge to simulate based on the scale factor from debye length
weight = actual_charge / (N_particles * q_p_real) # Weight calculation for both species of particle

# Weighted quantities for particles
q_e = q_e_real * weight
q_p = q_p_real * weight
m_e = m_e_real * weight
m_p = m_p_real * weight

In the first line of this section, I set $\lambda_D$ equal to twice the length of each grid cell. $\lambda_D$ is the Debye length, an important quantity in plasma simulations, which tells you how far away the electric field from one particle can influence another, before being weakened due to the influence of other surrounding particles[11]. I set $\lambda_D$ to twice the length of each grid cell to guarantee there are multiple cells per $\lambda_D$, because the opposite would cause problems with the accuracy of the simulation. Then, I rearrange this formula for $\lambda_D$:
$$\lambda_D = \sqrt{\frac{\epsilon_0 k_B T}{n q^2}}.$$
I solve for $n$, the number density of particles, so I can calculate the total number of particles I need to accurately simulate the calculated Debye length and plasma temperature. I calculate the amount of charge that needs to be simulated, use that to get the weighting factor for each particle, which I apply to the charge and mass of each simulated macroparticle. Applying this change immediately gave me the result I had been looking for, meaning that the problem with the simulation had been that there was not enough charge in the plasma for it to hold itself together.

![GIF of final simulation result](plasma_sim_20251203_152247.gif "Example of a result with macroparticle weighting")

The simulation result shown above is an example of what the final program produces after the introduction of macroparticle weighting. You can see the density of electrons slowly decrease around the edges of the plasma, eventually resulting in a strong electric field around the edge. The total number of real particles simulated can be calculated like so:

In [5]:
import numpy as np
# Simulation Parameters
L = 0.05
Nx = 64
dx = L / Nx
T_plasma = 1000
R_plasma = 0.0125
N_particles = int(1e5)
q_e_real = -1.602e-19
q_p_real = 1.602e-19
m_e_real = 9.109e-31
m_p_real = 1.672e-27
eps0 = 8.854e-12
kB = 1.380649e-23
# Macroparticle Parameters
lambda_D = 2 * dx
scale_n = (eps0 * kB * T_plasma)/(lambda_D**2 * q_e_real**2)
plasma_area = np.pi * R_plasma**2
actual_charge = scale_n * plasma_area * q_p_real
weight = actual_charge / (N_particles * q_p_real) 
# Weighted quantities for particles
q_e = q_e_real * weight
q_p = q_p_real * weight
m_e = m_e_real * weight
m_p = m_p_real * weight
print(f'Number of macroparticles: {N_particles} | Number of real particles: {int(actual_charge / q_p_real)} | Number of real particles per macroparticle: {int(weight)}')

Number of macroparticles: 100000 | Number of real particles: 957695889 | Number of real particles per macroparticle: 9576


This illustrates the benefits that come with the full PIC method, particularly the macroparticle weighting. I was able to run the simulation shown above, representing the behavior of nearly one billion particles, in less than a minute. Attempting to run the code for one billion particles without weighting would be a disaster; it would take a ridiculous amount of time, if it ran at all. The method does come with some downsides, however. You can see during the initial part of the simulation that there are fluctuations in the electric field strength throughout the plasma, instead of a uniform, very low strength field as you might expect from an electrically neutral plasma. This error is a result of the PIC methods, where charges are assigned to grids, and the calculations are performed on the grid points, rather than on the continuous field of potential, charge density, etc. The error could be improved, but it would require more complex computational methods or a higher number of macroparticles. I ran the simulation shown below with one million macroparticles, and as you can see, it did not result in a significant improvement in the noise in the electric field.

![Example of simulation with 1e6 macroparticles](plasma_sim_20251204_121427.gif "Example of a simulation with 1e6 macroparticles")

## Conclusion

My program demonstrates the formation of a sheath along the boundary of an electrically neutral, collisionless plasma in a vacuum. Understanding the behavior of sheaths is important for many applications of plasma physics, including nuclear fusion and plasma propulsion engines. The particle in cell methods allow for efficient simulation of very large numbers of particles without requiring large amounts of computational power (I ran these simulations on a laptop, and, excluding the simulation with one million macroparticles, they completed in less than a minute each). The nature of the grid methods used does introduce some error into the calculations and the results, but it does not significantly alter the outcome of the simulations, and it could be improved through a variety of changes, such as using a larger number of macroparticles, using a smaller grid cell size, or using higher-order interpolation methods. Some further improvements to the program could include using those methods that would decrease the error, increasing the resolution of the plots in the output animation, saving the simulation data to csv files for future use, calculating and displaying the real time that the simulation took to run, and calculating and displaying the total timespan of the simulation to increase clarity about the timescale shown.

## References

[1]	“About Plasmas and Fusion,” Princeton Plasma Physics Labratory. [Online]. Available: https://www.pppl.gov/about/about-plasmas-and-fusion. [Accessed: 03-Dec-2025].

[2]	“Plasma, the fourth state of matter,” Pie Scientific. [Online]. Available: https://piescientific.com/resource_pages/resource_introduction_to_plasma/. [Accessed: 03-Dec-2025].

[3]	J. Langlois, “PIC Simulation - A 2D plasma sheath formation,” YouTube, 21-Jan-2021. [Online]. Available: https://www.youtube.com/watch?v=SJUyCjAwrTI. [Accessed: 16-Oct-2025].

[4]	“The Electrostatic Particle In Cell (ES-PIC) Method,” pic-c, 08-Nov-2010. [Online]. Available: https://www.particleincell.com/2010/es-pic-method/. [Accessed: 03-Dec-2025].

[5]	“The Maxwell-Boltzmann distribution,” Royal Holloway, University of London. [Online]. Available: https://www.pp.rhul.ac.uk/~cowan/ph290/notes/mb_distribution.pdf.

[6]	P. E. Fleming, “The Gaussian Distribution of One Component of the Molecular Velocity,” LibreTexts Chemistry, 08-Mar-2025. [Online]. Available: https://chem.libretexts.org/@go/page/14536. [Accessed: 03-Dec-2025].

[7]	“Random Uniformly Distributed Points in a Circle,” Math StackExchange, 2015. [Online]. Available: https://math.stackexchange.com/questions/1307287/random-uniformly-distributed-points-in-a-circle. [Accessed: 03-Dec-2025].

[8]	U. Ravaioli, “Introduction to the Numerical Solution of Partial Differential Equations,” University of Illinois Urbana-Champaign. [Online]. Available: https://transport.ece.illinois.edu/ECE539S12-Lectures/Chapter1-NumericalMethods.pdf. [Accessed: 03-Dec-2025].

[9]	“Bilinear image interpolation / scaling - A calculation example,” StackOverflow, 2015. [Online]. Available: https://stackoverflow.com/questions/32124170/bilinear-image-interpolation-scaling-a-calculation-example. [Accessed: 03-Dec-2025].

[10]	J. Schloss, “Verlet Integration,” Algorithm Archive. [Online]. Available: https://www.algorithm-archive.org/contents/verlet_integration/verlet_integration.html. [Accessed: 03-Dec-2025].

[11]	“Debye Length,” Impedans Plasma Measurement, 29-May-2024. [Online]. Available: https://www.impedans.com/docs/debye-length/. [Accessed: 03-Dec-2025].


